In [96]:
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy import optimize
import sympy as sm
from scipy.optimize import minimize 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import numpy as np
from scipy.optimize import minimize


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Analytical solution using Sumpy

To study the fertility model, we need some variables that define income, consumption, population and more.

In [135]:
#Defining variables and parameters
Y = sm.symbols('Y_t')
alpha = sm.symbols('alpha')
L = sm.symbols('L_t')
A = sm.symbols('A')
X = sm.symbols('X')
n = sm.symbols('n_t')
lambda_ = sm.symbols('lambda')
tau = sm.symbols('tau')
beta= sm.symbols('beta')
mu = sm.symbols('mu')
c = sm.symbols('c_t')
y = sm.symbols('y_t')
u= sm.symbols('u_t')
L1 = sm.symbols('L_t+1')
e = sm.symbols('eta')
L_ss = sm.symbols('L^*')

Writing up the utility

In [98]:
# Defining utility function
utility = beta*sm.ln(c)+(1-beta)*sm.ln(n)
utility

beta*log(c_t) + (1 - beta)*log(n_t)

The problem is to choose values for c and n such that the utility maximizes, given the following:

In [99]:
budgetconstraint = sm.Eq(c+lambda_*n,y*(1-tau))
budgetconstraint

Eq(c_t + lambda*n_t, y_t*(1 - tau))

Now solving with respect to c, which is the number of surviving children

In [100]:
c_solve = sm.solve(budgetconstraint, c)
c_substituted = c_solve[0]
c_substituted

-lambda*n_t - tau*y_t + y_t

Inserting this in the utility function to make it a function of n, we can maximize given the equation below

In [101]:
utility_substituted = utility.subs(c, c_substituted)
utility_substituted

beta*log(-lambda*n_t - tau*y_t + y_t) + (1 - beta)*log(n_t)

Taking the first order condition with respect to n to find the optimal level of births

In [130]:
first_order = sm.diff(utility_substituted, n)

first_order

-beta*lambda/(-lambda*n_t - tau*y_t + y_t) + (1 - beta)/n_t

The optimal number of births are then:

In [137]:
pop = sm.Eq(e, ((1-beta)/(lambda_))*(1-tau))
pop

Eq(eta, (1 - beta)*(1 - tau)/lambda)

In [103]:
n_solve = sm.solve(first_order, n)
n_solve[0]

y_t*(beta*tau - beta - tau + 1)/lambda

In [145]:
n_solvedd = sm.Eq(n, n_solve[0])
n_solvedd

Eq(n_t, y_t*(beta*tau - beta - tau + 1)/lambda)

In [152]:
sovledddd = n_solvedd.subs(e, (((1-beta)/(lambda_))*(1-tau)))
sovledddd

Eq(n_t, y_t*(beta*tau - beta - tau + 1)/lambda)

Going further, we look at the transition equation

In [104]:
transition = sm.Eq(n*L+(1-mu)*L, L1)
transition

Eq(L_t*n_t + L_t*(1 - mu), L_t+1)

Inserting per capita production equation into the fertility equation assuming:

In [105]:
income1 = sm.Eq(y, ((A*X)/L)**alpha)
income1

Eq(y_t, (A*X/L_t)**alpha)

In [106]:
transition_1 = transition.subs(n, n_solve[0])
transition_1

Eq(L_t*(1 - mu) + L_t*y_t*(beta*tau - beta - tau + 1)/lambda, L_t+1)

So, we get:

Using this term in the transition equation to get:

In [107]:
transition_2 = transition_1.subs(y, income1.rhs)
transition_2

Eq(L_t*(1 - mu) + L_t*(A*X/L_t)**alpha*(beta*tau - beta - tau + 1)/lambda, L_t+1)

Now we assume steady state levels $$L_{t+1}=L_t=L^*$$

In [108]:
L_steadystate = sm.Eq(L, L_ss)
L_steadystate

Eq(L_t, L^*)

In [109]:
transition_3 = transition_2.subs(L, L_steadystate.rhs)
transition_4 = transition_3.subs(L1, L_steadystate.rhs)
transition_4

Eq(L^**(1 - mu) + L^**(A*X/L^*)**alpha*(beta*tau - beta - tau + 1)/lambda, L^*)

Solving the steady state level for population:

In [132]:
transition_5 = sm.solve(sm.Eq(transition_4.lhs,0), L_steadystate.rhs)
transition_5[0]

A*X/(lambda*(mu - 1)/((beta - 1)*(tau - 1)))**(1/alpha)

We have the steady state for population and can therefore insert it in the income equation to get the steady state income level $$y^*=\left(\frac{AX}{L^*}\right)^\alpha$$
Inserting the steady state level of labor will give the following
$$y^*=\left(\frac{AX}{AX\left(\frac{\lambda(\mu-1)}{(\beta-1)(\tau-1)}\right)^{-\frac{1}{\alpha}}}\right)^\alpha$$
Rewriting this we get:
$$y^*=\left(\left(\frac{\lambda(\mu-1)}{(\beta-1)(\tau-1)}\right)^{\frac{1}{\alpha}}\right)^\alpha$$

The following code below will prove this:

In [114]:
income2 = income1.subs(L, transition_5[0])
income2

Eq(y_t, ((lambda*(mu - 1)/((beta - 1)*(tau - 1)))**(1/alpha))**alpha)

The power cancels out and we can write it up as:
$$y^*=\frac{\lambda(\mu-1)}{(\beta-1)(\tau-1)}$$

For the parameters to be used in a computational way, they need to be converted using the lambdify option from sympy.

In [127]:
steadystate_income = sm.lambdify((alpha, beta,lambda_,mu,tau), income2)

Assuming the following parameter values:
- $\alpha = 0.5$
- $\beta = 0.5$
- $\lambda = 1$
- $\mu = 0.01$
- $\tau = 0.4$

In [128]:
steadystate_income(0.5,0.5, 0.1, 2, 0.4)

False